In [ ]:
import requests
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import yfinance as yf

# Initialize LLM (replace with actual API keys)
llm = OpenAI(temperature=0.7)

# Set up Memory for Multi-turn Interaction
memory = ConversationBufferMemory()

# Define Prompt Template
financial_prompt = PromptTemplate(
    input_variables=["news_sentiment", "economic_factors", "high_impact_news", "related_stories"],
    template="""
    Based on the following financial information, provide actionable trading insights:
    - News Sentiment: {news_sentiment}
    - Economic Factors: {economic_factors}
    - High-Impact News: {high_impact_news}
    - Related Stories: {related_stories}

    Identify potential buy and sell opportunities and the rationale behind them.
    """
)

# Create LangChain Chain
financial_chain = LLMChain(llm=llm, prompt=financial_prompt, memory=memory)

# Function to Fetch Similar Stories from MarketAux API
def fetch_similar_stories(uuid, api_token, sentiment_gte=None, sentiment_lte=None, limit=3):
    base_url = "https://api.marketaux.com/v1/news/similar"
    params = {
        "uuid": uuid,
        "api_token": api_token,
        "sentiment_gte": sentiment_gte,
        "sentiment_lte": sentiment_lte,
        "limit": limit,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Process Similar Stories Data
def process_similar_stories(similar_response):
    if not similar_response or "data" not in similar_response:
        return [], 0

    stories = similar_response["data"]
    related_stories = []
    sentiment_scores = []

    for story in stories:
        title = story["title"]
        sentiment_score = story.get("entities", [])[0].get("sentiment_score", 0) if story.get("entities") else 0
        related_stories.append(f"{title} (Sentiment: {sentiment_score})")
        sentiment_scores.append(sentiment_score)

    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
    return related_stories, avg_sentiment

# Fetch Macroeconomic Data (Example)

def get_economic_factors():
    """
    Fetches proxies for GDP, inflation rate, and unemployment rate using Yahoo Finance.
    Note: Yahoo Finance does not directly provide these indicators, so this uses related indices.
    """
    print("---Fetch Real-Time Economic Data Node---")
    try:
        # Fetch proxies for economic indicators
        gdp_proxy = yf.Ticker("^GSPC").info  # S&P 500 Index as a GDP growth proxy
        inflation_proxy = yf.Ticker("^IRX").info  # Treasury Yield (13 Week) as inflation proxy
        unemployment_proxy = yf.Ticker("^DJI").info  # Dow Jones Industrial Average for employment trends

        # Process the proxies into understandable values
        economic_data = {
            "GDP growth (proxy - S&P 500 YoY)": gdp_proxy.get("regularMarketPreviousClose", "N/A"),
            "Inflation rate (proxy - 13W Yield)": inflation_proxy.get("regularMarketPreviousClose", "N/A"),
            "Unemployment rate (proxy - Dow YoY)": unemployment_proxy.get("regularMarketPreviousClose", "N/A"),
        }
    except Exception as e:
        print(f"Error fetching economic data: {e}")
        economic_data = {
            "GDP growth": "Error",
            "Inflation rate": "Error",
            "Unemployment rate": "Error",
        }

    print(f"Economic Factors: {economic_data}")
    return economic_data


# Main Integration
if __name__ == "__main__":
    # Fetch Real-Time Economic Factors
    economic_factors = get_economic_factors()

    # Simulated High-Impact News
    high_impact_news = [
        "Trump wins 2024, markets surge globally (Sentiment: 0.7783)",
        "Amazon.com mulls new multi-billion dollar investment in Anthropic (Sentiment: 0)",
    ]

    # Fetch Similar Stories
    primary_article_uuid = "70cb577e-c2dd-4dde-b501-f713823a4939"
    api_token = MARKET_AUX  # Replace with your API token
    similar_response = fetch_similar_stories(
        uuid=primary_article_uuid,
        api_token=api_token,
        sentiment_gte=0  # Filter for neutral/positive sentiment
    )
    related_stories, avg_related_sentiment = process_similar_stories(similar_response)

    # Generate Trading Insights
    trading_insights = financial_chain.run(
        news_sentiment=avg_related_sentiment,
        economic_factors="\n".join(f"{key}: {value}" for key, value in economic_factors.items()),
        high_impact_news="\n".join(high_impact_news),
        related_stories="\n".join(related_stories)
    )

    # Display Results
    print("\nTrading Insights:\n", trading_insights)

    print("\nRelated Stories:\n")
    for story in related_stories:
        print(f"- {story}")



In [4]:
# Install LangChain
!pip install langchain
# Install psycopg2 for PostgreSQL database interaction
!pip install psycopg2-binary
# Install requests for API calls
!pip install requests
# Install yfinance for financial data
!pip install yfinance
!pip install langchain-google-genai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00


In [7]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI # Changed import
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import psycopg2
from psycopg2.extras import RealDictCursor
import requests
import yfinance as yf

# Set environment variables for API keys
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["POSTGRES_API_KEY"] = userdata.get("POSTGRES_DB_01")
os.environ["MARKETAUX_API_KEY"] = userdata.get("MARKET_AUX")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"
MARKET_AUX = os.getenv("MARKETAUX_API_KEY")


# Database connection setup
def connect_to_db():
    db_url = os.getenv("POSTGRES_DB_01")
    try:
        conn = psycopg2.connect(db_url, sslmode='require')
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None

# LLM Configuration
def create_llm_chain(prompt_template, temp=0.7, max_tokens=150):
    llm = ChatGoogleGenerativeAI(  # Changed to ChatGoogleGenerativeAI
        google_api_key=os.getenv("GEMINI_API_KEY"),
        model="gemini-pro",
        temperature=temp,
        max_tokens=max_tokens,
    )
    chain = LLMChain(llm=llm, prompt=prompt_template)
    return chain

# Function to fetch similar stories from MarketAux API
def fetch_similar_stories(uuid, api_token, sentiment_gte=None, sentiment_lte=None, limit=3):
    base_url = "https://api.marketaux.com/v1/news/similar"
    params = {
        "uuid": uuid,
        "api_token": api_token,
        "sentiment_gte": sentiment_gte,
        "sentiment_lte": sentiment_lte,
        "limit": limit,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Process similar stories
def process_similar_stories(similar_response):
    if not similar_response or "data" not in similar_response:
        return [], 0

    stories = similar_response["data"]
    related_stories = []
    sentiment_scores = []

    for story in stories:
        title = story["title"]
        sentiment_score = story.get("entities", [])[0].get("sentiment_score", 0) if story.get("entities") else 0
        related_stories.append(f"{title} (Sentiment: {sentiment_score})")
        sentiment_scores.append(sentiment_score)

    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
    return related_stories, avg_sentiment

# Function to fetch macroeconomic factors
def get_economic_factors():
    try:
        gdp_proxy = yf.Ticker("^GSPC").info
        inflation_proxy = yf.Ticker("^IRX").info
        unemployment_proxy = yf.Ticker("^DJI").info

        economic_data = {
            "GDP growth (proxy - S&P 500 YoY)": gdp_proxy.get("regularMarketPreviousClose", "N/A"),
            "Inflation rate (proxy - 13W Yield)": inflation_proxy.get("regularMarketPreviousClose", "N/A"),
            "Unemployment rate (proxy - Dow YoY)": unemployment_proxy.get("regularMarketPreviousClose", "N/A"),
        }
    except Exception as e:
        print(f"Error fetching economic data: {e}")
        economic_data = {
            "GDP growth": "Error",
            "Inflation rate": "Error",
            "Unemployment rate": "Error",
        }

    return economic_data

# Function to fetch asset-specific data
def fetch_asset_data(asset_type, symbol):
    try:
        data = yf.Ticker(symbol).info
        asset_data = {
            "Asset": symbol,
            "Current Price": data.get("regularMarketPrice", "N/A"),
            "52-Week High": data.get("fiftyTwoWeekHigh", "N/A"),
            "52-Week Low": data.get("fiftyTwoWeekLow", "N/A"),
            "Market Cap": data.get("marketCap", "N/A"),
            "P/E Ratio": data.get("trailingPE", "N/A"),
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        asset_data = {
            "Asset": symbol,
            "Current Price": "Error",
            "52-Week High": "Error",
            "52-Week Low": "Error",
            "Market Cap": "Error",
            "P/E Ratio": "Error",
        }

    return asset_data

def fetch_articles_and_extract_uuid(api_token, symbols=None, industries=None, limit=5):
    base_url = "https://api.marketaux.com/v1/news/all"
    params = {
        "api_token": api_token,
        "symbols": symbols,  # e.g., "AAPL" or "BTC-USD"
        "industries": industries,  # e.g., "Technology"
        "limit": limit,
    }
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            articles = response.json().get("data", [])
            if not articles:
                print("No articles found.")
                return []

            # Extract UUIDs and titles
            extracted_uuids = [
                {"uuid": article["uuid"], "title": article["title"]} for article in articles
            ]
            print(f"Extracted UUIDs: {extracted_uuids}")
            return extracted_uuids
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return []
    except Exception as e:
        print(f"Error fetching articles: {e}")
        return []


# Define Financial Prompt Template
financial_prompt = PromptTemplate(
    input_variables=["news_sentiment", "economic_factors", "high_impact_news", "related_stories", "asset_data"],
    template="""Based on the following financial information, provide actionable trading insights for the selected asset:
    - Asset Data: {asset_data}
    - News Sentiment: {news_sentiment}
    - Economic Factors: {economic_factors}
    - High-Impact News: {high_impact_news}
    - Related Stories: {related_stories}

    Identify potential buy and sell opportunities and the rationale behind them.
    """
)

# Financial Analysis Chain
def financial_analysis(asset_type, symbol, news_sentiment, economic_factors, high_impact_news, related_stories):
    asset_data = fetch_asset_data(asset_type, symbol)
    memory = ConversationBufferMemory()
    chain = create_llm_chain(financial_prompt, temp=0.7, max_tokens=200)
    inputs = {
        "asset_data": asset_data,
        "news_sentiment": news_sentiment,
        "economic_factors": economic_factors,
        "high_impact_news": high_impact_news,
        "related_stories": related_stories,
    }
    response = chain.run(inputs)
    return response

# Main Function
if __name__ == "__main__":
    print("Financial Analyst Tool")

    # Sample inputs
    # example_uuid = "sample_uuid"  # Replace with actual UUID
    marketaux_api_token = os.getenv("MARKETAUX_API_KEY") #fixed variable name
    asset_type = "crypto"  # User-selected asset type (e.g., "currency pair", "indices", etc.)
    symbol = "BTC-USD"  # User-selected symbol (e.g., "AAPL", "EURUSD", etc.)

    # Fetch articles and extract UUIDs
    extracted_uuids = fetch_articles_and_extract_uuid(
        api_token=marketaux_api_token,
        symbols=symbol,
        industries=asset_type,
        limit=3
    )

    if not extracted_uuids:
      print("No UUIDs available. Skipping similar stories.")
      example_uuid = None
    else:
        # Use the first UUID for analysis (or loop through all UUIDs if required)
        example_uuid = extracted_uuids[0]["uuid"]
        print(f"Using UUID: {example_uuid} for similar story analysis.")

    # Fetch economic factors
    economic_factors = get_economic_factors()

    if example_uuid:
        # Fetch similar stories
        similar_stories_response = fetch_similar_stories(example_uuid, marketaux_api_token)
        related_stories, avg_sentiment = process_similar_stories(similar_stories_response)
    else:
        related_stories, avg_sentiment = [], 0

    # High-impact news (mocked for example)
    high_impact_news = "Federal Reserve announces interest rate hike."

    # Financial Analysis
    insights = financial_analysis(
        asset_type=asset_type,
        symbol=symbol,
        news_sentiment=f"Average Sentiment: {avg_sentiment}",
        economic_factors=economic_factors,
        high_impact_news=high_impact_news,
        related_stories=related_stories
    )

    print("\nActionable Insights:")
    print(insights)

Financial Analyst Tool
No articles found.
No UUIDs available. Skipping similar stories.

Actionable Insights:
**Buy Opportunity:**

* **Rationale:** The Federal Reserve's interest rate hike could potentially weaken the US dollar, making Bitcoin more attractive as a hedge against inflation.
* **Potential Entry Point:** Current price is not available, but if it falls below the 52-week low, it could present a potential buy opportunity.

**Sell Opportunity:**

* **Rationale:** The average news sentiment is neutral, and there is no significant high-impact news that could drive the price higher.
* **Potential Exit Point:** If the price approaches the 52-week high, it could be an indication to sell.

**Additional Considerations:**

* **Market Cap:** Bitcoin's large market cap suggests it has a high level of liquidity and stability.
* **Economic Factors:** The proxy GDP growth and inflation rate are high, indicating a strong economy. However, the proxy unemployment rate is also high, which cou

In [2]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
